In [1]:
import pandas as pd
import warnings
import sys
import numpy as np
import scipy as sc
import random
from scipy import stats
from numpy import log10, sqrt

mydir = '/Users/kenlocey/GitHub/HACRP-HAIs/'
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## Load file from Part 1

In [2]:
df_2017 = pd.read_pickle('~/GitHub/HACRP-HAIs/data/merged_HAC_HAI/P1_HAI_HAC_2017.pkl')

## Separate incorrect penalty assignments from correct assignments

In [3]:
err_df = df_2017[df_2017['Payment Reduction Reproduced?'] == 0]
df_2017 = df_2017[~df_2017['Facility ID'].isin(err_df['Facility ID'])]

err_df.head()

,AHRQ PSI-90 Footnote,AHRQ PSI-90 Score,CAUTI Footnote,CAUTI Score,CDI Footnote,CDI Score,CLABSI Footnote,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Score Footnote,Domain 1 Start Date,End Date,Domain 2 Score,Domain 2 Score Footnote,Start Date,Facility ID,Facility Name,Fiscal Year,MRSA Footnote,MRSA Score,Payment Reduction,Payment Reduction Footnote,SSI Footnote,SSI Score,State,Total HAC Footnote,Total HAC Score,file_month,file_year,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,MRSA patient days,CDI patient days,CAUTI Observed Cases,CLABSI Observed Cases,MRSA Observed Cases,CDI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,MRSA Predicted Cases,CDI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,MRSA derived SIR,CDI derived SIR,CAUTI derived score,CLABSI derived score,MRSA derived score,CDI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CDI delta,MRSA delta,CAUTI delta,CLABSI delta
2247,NaN,4.0,NaN,5.0,NaN,9.0,NaN,8.0,6302015,4.0,NaN,7012013,2015-12-31,7.2,NaN,2014-01-01,360180,CLEVELAND CLINIC,2017,NaN,9.0,Yes,NaN,NaN,5.0,OH,NaN,6.72,10,2017,208865.0,102625.0,106240.0,724745.0,716275.0,138.0,125.0,75.0,761.0,221.751,192.892,61.825,661.323,0.622320,0.648031,1.213101,1.150724,5.0,8.0,8.0,8.0,6.38,No,0,0.34,1.0,1.0,0.0,0.0
2143,5.0,NaN,5.0,NaN,NaN,4.0,5.0,NaN,6302015,NaN,5.0,7012013,2015-12-31,4.0,NaN,2014-01-01,340187,DAVIE MEDICAL CENTER,2017,5.0,NaN,No,NaN,5.0,NaN,NC,NaN,4.00,10,2017,0.0,0.0,0.0,7100.0,7100.0,0.0,0.0,0.0,3.0,0.000,0.000,0.212,2.443,NaN,NaN,0.000000,1.227998,NaN,NaN,NaN,9.0,9.00,Yes,0,-5.00,-5.0,NaN,NaN,NaN
2403,NaN,9.0,NaN,8.0,NaN,5.0,NaN,5.0,6302015,9.0,NaN,7012013,2015-12-31,6.0,NaN,2014-01-01,390001,GEISINGER - COMMUNITY MEDICAL CENTER,2017,NaN,4.0,No,NaN,NaN,8.0,PA,NaN,6.45,10,2017,11723.0,6650.0,5073.0,105887.0,105887.0,9.0,3.0,5.0,9.0,8.383,7.566,7.580,11.244,1.073601,0.396511,0.659631,0.800427,8.0,5.0,5.0,5.0,6.62,Yes,0,-0.17,0.0,-1.0,0.0,0.0
1788,NaN,8.0,NaN,5.0,NaN,9.0,NaN,6.0,6302015,8.0,NaN,7012013,2015-12-31,6.4,NaN,2014-01-01,290039,MOUNTAINVIEW HOSPITAL,2017,NaN,7.0,Yes,NaN,NaN,5.0,NV,NaN,6.64,10,2017,38660.0,18865.0,19795.0,201077.0,189350.0,28.0,18.0,8.0,184.0,44.762,37.970,11.048,148.018,0.625531,0.474058,0.724113,1.243092,5.0,6.0,5.0,9.0,6.30,No,0,0.34,0.0,2.0,0.0,0.0
1693,NaN,2.0,NaN,7.0,NaN,9.0,NaN,7.0,6302015,2.0,NaN,7012013,2015-12-31,7.4,NaN,2014-01-01,260068,BOONE HOSPITAL CENTER,2017,NaN,8.0,Yes,NaN,NaN,6.0,MO,NaN,6.59,10,2017,19608.0,11387.0,8221.0,141163.0,126269.0,12.0,7.0,7.0,120.0,13.665,12.829,6.970,100.059,0.878156,0.545639,1.004304,1.199292,7.0,7.0,7.0,9.0,6.42,No,0,0.17,0.0,1.0,0.0,0.0


## Attempt to correct predicted cases

In [4]:
np.random.seed(0)

prvdrs = err_df['Facility ID'].tolist()

derived_predicted_cdi = []
derived_predicted_mrsa = []

for i, prvdr in enumerate(prvdrs):
    
    print('CMS #:', prvdr)
    ####################
    ####  CDI  #########
    ####################
    
    # delta is reported - derived
    # negative delta: reported < derived; means that predicted cases needs to be increased
    # positive delta: reported > derived; means that predicted cases needs to be decreased
        
    d = err_df['CDI delta'].iloc[i]
    o = err_df['CDI Observed Cases'].iloc[i]
    p = err_df['CDI Predicted Cases'].iloc[i]
    reported_score = err_df['CDI Score'].iloc[i]
    derived_score = err_df['CDI derived score'].iloc[i]
    
    print('CDI')
    print('obs:', o)
    print('pred:', p)
    print('delta:', d)
    print('reported_score:', reported_score)
    print('derived_score:', derived_score)
    
    n = 0
    score = float(derived_score)
    
    if score == reported_score:
        derived_predicted_cdi.append(p)
    else:
        while score != reported_score:

            if o == 0:
                print('CDI:', len(prvdrs) - i)
                derived_predicted_cdi.append(p)
                break

            if reported_score - score < 0:
                n += np.random.uniform(0, 1)

            elif reported_score - score > 0:
                n -= np.random.uniform(0, 1)

            n_p = p + n    
            sir = o/n_p

            if sir < 0.168:
                score = 1
            elif sir <= 0.425:
                score = 2
            elif sir <= 0.582:
                score = 3
            elif sir <= 0.705:
                score = 4
            elif sir <= 0.807:
                score = 5
            elif sir <= 0.915:
                score = 6
            elif sir <= 1.031:
                score = 7
            elif sir <= 1.173:
                score = 8
            elif sir <= 1.382:
                score = 9
            elif sir > 1.382:
                score = 10
            else:
                score = np.nan

            if score == reported_score:
                print('CDI:', len(prvdrs) - i)
                derived_predicted_cdi.append(n_p)
    
    
    ####################
    ####  MRSA  ########
    ####################
    
    # delta is reported - derived
    # negative delta: reported < derived; means that predicted cases needs to be increased
    # positive delta: reported > derived; means that predicted cases needs to be decreased
        
    d = err_df['MRSA delta'].iloc[i]
    o = err_df['MRSA Observed Cases'].iloc[i]
    p = err_df['MRSA Predicted Cases'].iloc[i]
    reported_score = err_df['MRSA Score'].iloc[i]
    derived_score = err_df['MRSA derived score'].iloc[i]
    
    print('MRSA')
    print('obs:', o)
    print('pred:', p)
    print('delta:', d)
    print('reported_score:', reported_score)
    print('derived_score:', derived_score)
    
    n = 0
    score = float(derived_score)
    
    if score == reported_score:
        derived_predicted_mrsa.append(p)
    else:
        while score != reported_score:

            if o == 0:
                print('MRSA:', len(prvdrs) - i)
                derived_predicted_mrsa.append(p)
                break

            if reported_score - score < 0:
                n += np.random.uniform(0, 2)

            elif reported_score - score > 0:
                n -= np.random.uniform(0, 2)

            n_p = p + n    
            sir = o/n_p

            if sir == 0.0:
                score = 1
            elif sir <= 0.298:
                score = 2
            elif sir <= 0.478:
                score = 3
            elif sir <= 0.623:
                score = 4
            elif sir <= 0.763:
                score = 5
            elif sir <= 0.923:
                score = 6
            elif sir <= 1.104:
                score = 7
            elif sir <= 1.344:
                score = 8
            elif sir <= 1.722:
                score = 9
            elif sir > 1.722:
                score = 10
            else:
                score = np.nan

            if score == reported_score:
                print('MRSA:', len(prvdrs) - i, '\n')
                derived_predicted_mrsa.append(n_p)
        
    
print(len(derived_predicted_mrsa))
print(len(derived_predicted_cdi))
print(err_df.shape)


err_df['MRSA Predicted Cases'] = derived_predicted_mrsa
err_df['CDI Predicted Cases'] = derived_predicted_cdi
err_df['MRSA derived SIR'] = err_df['MRSA Observed Cases'] / err_df['MRSA Predicted Cases']
err_df['CDI derived SIR'] = err_df['CDI Observed Cases'] / err_df['CDI Predicted Cases']


CMS #: 360180
CDI
obs: 761.0
pred: 661.3230000000001
delta: 1.0
reported_score: 9.0
derived_score: 8.0
CDI: 43
MRSA
obs: 75.0
pred: 61.825
delta: 1.0
reported_score: 9.0
derived_score: 8.0
MRSA: 43 

CMS #: 340187
CDI
obs: 3.0
pred: 2.443
delta: -5.0
reported_score: 4.0
derived_score: 9.0
CDI: 42
MRSA
obs: 0.0
pred: 0.212
delta: nan
reported_score: nan
derived_score: nan
MRSA: 42
CMS #: 390001
CDI
obs: 9.0
pred: 11.244
delta: 0.0
reported_score: 5.0
derived_score: 5.0
MRSA
obs: 5.0
pred: 7.58
delta: -1.0
reported_score: 4.0
derived_score: 5.0
MRSA: 41 

CMS #: 290039
CDI
obs: 184.0
pred: 148.018
delta: 0.0
reported_score: 9.0
derived_score: 9.0
MRSA
obs: 8.0
pred: 11.048
delta: 2.0
reported_score: 7.0
derived_score: 5.0
MRSA: 40 

CMS #: 260068
CDI
obs: 120.0
pred: 100.059
delta: 0.0
reported_score: 9.0
derived_score: 9.0
MRSA
obs: 7.0
pred: 6.970000000000001
delta: 1.0
reported_score: 8.0
derived_score: 7.0
MRSA: 39 

CMS #: 260025
CDI
obs: 8.0
pred: 14.152999999999999
delta: 1.0
repo

In [5]:
df_2017 = pd.concat([df_2017, err_df])

## Reproduce HAC scores and penalty assignments for 2017

In [6]:

df_2017.dropna(how='all', axis=1, inplace=True)
df_2017 = df_2017[~df_2017['Total HAC Score'].isin([float("NaN"), np.nan])]
df_2017 = df_2017[df_2017['State'] != 'MD']
df_2017 = df_2017[df_2017['Payment Reduction'].isin(['Yes', 'No'])]

hais = ['CAUTI', 'CLABSI', 'MRSA', 'CDI']
start_dates = df_2017['Start Date'].unique()

new_df = 0
    
for hai in hais:
    tdf2 = df_2017[~df_2017[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    if tdf2.shape[0] > 0:
            
        derived_sirs = tdf2[hai + ' derived SIR'].tolist()
        reported_scores = tdf2[hai + ' Score'].tolist()
        derived_scores = []

        for ii, sir in enumerate(derived_sirs):
            if np.isnan(sir) == True or np.isnan(reported_scores[ii]) == True:
                derived_scores.append(np.nan)
            elif hai == 'CLABSI':
                score = np.nan
                if sir == 0.0:
                    score = 1
                elif sir <= 0.125:
                    score = 2
                elif sir <= 0.24:
                    score = 3
                elif sir <= 0.325:
                    score = 4
                elif sir <= 0.4055:
                    score = 5
                elif sir <= 0.502:
                    score = 6
                elif sir <= 0.613:
                    score = 7
                elif sir <= 0.753:
                    score = 8
                elif sir <= 0.992:
                    score = 9
                elif sir > 0.992:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)
                
            elif hai == 'CAUTI':
                score = np.nan
                if sir == 0.0:
                    score = 1
                elif sir <= 0.222:
                    score = 2
                elif sir <= 0.407:
                    score = 3
                elif sir <= 0.558:
                    score = 4
                elif sir <= 0.697:
                    score = 5
                elif sir <= 0.845:
                    score = 6
                elif sir <= 1.035:
                    score = 7
                elif sir <= 1.261:
                    score = 8
                elif sir <= 1.596:
                    score = 9
                elif sir > 1.596:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)
                
            
            elif hai == 'MRSA':
                score = np.nan
                if sir == 0.0:
                    score = 1
                elif sir <= 0.298:
                    score = 2
                elif sir <= 0.478:
                    score = 3
                elif sir <= 0.623:
                    score = 4
                elif sir <= 0.763:
                    score = 5
                elif sir <= 0.923:
                    score = 6
                elif sir <= 1.104:
                    score = 7
                elif sir <= 1.344:
                    score = 8
                elif sir <= 1.722:
                    score = 9
                elif sir > 1.722:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)
                
                
            elif hai == 'CDI':
                score = np.nan
                if sir < 0.168:
                    score = 1
                elif sir <= 0.425:
                    score = 2
                elif sir <= 0.582:
                    score = 3
                elif sir <= 0.705:
                    score = 4
                elif sir <= 0.807:
                    score = 5
                elif sir <= 0.915:
                    score = 6
                elif sir <= 1.031:
                    score = 7
                elif sir <= 1.173:
                    score = 8
                elif sir <= 1.382:
                    score = 9
                elif sir > 1.382:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)


        tdf2[hai + ' derived score'] = derived_scores
        print('len:', len(derived_scores), len(reported_scores))

    # Assign maximum scores to hospitals with HAI footnote 18 
    tdf3 = df_2017[df_2017[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    if tdf3.shape[0] > 0:
        tdf3[hai + ' derived score'] = 10
        df_2017 = pd.concat([tdf2, tdf3], axis=0)
            
    else:
        df_2017 = tdf2.copy(deep=True)

features = ['CAUTI derived SIR', 'CAUTI Score', 'CAUTI derived score',
            'CLABSI derived SIR', 'CLABSI Score', 'CLABSI derived score',
            'SSI Score', 'AHRQ PSI-90 Score',
            'CDI derived SIR', 'CDI Score', 'CDI derived score',
            'MRSA derived SIR', 'MRSA Score', 'MRSA derived score',
           ]

for f in features:
    df_2017[f] = pd.to_numeric(df_2017[f], errors='coerce')

df_2017.head()

len: 3186 3186
len: 3186 3186
len: 3180 3180
len: 3180 3180


,AHRQ PSI-90 Footnote,AHRQ PSI-90 Score,CAUTI Footnote,CAUTI Score,CDI Footnote,CDI Score,CLABSI Footnote,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Score Footnote,Domain 1 Start Date,End Date,Domain 2 Score,Domain 2 Score Footnote,Start Date,Facility ID,Facility Name,Fiscal Year,MRSA Footnote,MRSA Score,Payment Reduction,SSI Footnote,SSI Score,State,Total HAC Footnote,Total HAC Score,file_month,file_year,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,MRSA patient days,CDI patient days,CAUTI Observed Cases,CLABSI Observed Cases,MRSA Observed Cases,CDI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,MRSA Predicted Cases,CDI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,MRSA derived SIR,CDI derived SIR,CAUTI derived score,CLABSI derived score,MRSA derived score,CDI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CDI delta,MRSA delta,CAUTI delta,CLABSI delta
0,NaN,1.0,NaN,10.0,NaN,7.0,NaN,10.0,6302015,1.0,NaN,7012013,2015-12-31,8.0,NaN,2014-01-01,010001,SOUTHEAST ALABAMA MEDICAL CENTER,2017,NaN,6.0,Yes,NaN,7.0,AL,NaN,6.95,10,2017,18260.0,13811.0,4449.0,182743.0,176802.0,50.0,18.0,12.0,94.0,16.583,6.676,14.625,118.846,3.015136,2.696225,0.820513,0.790940,10.0,10.0,6.0,5.0,6.61,Yes,1,3.400000e-01,2.0,0.0,0.0,0.0
2203,NaN,8.0,NaN,4.0,NaN,6.0,NaN,1.0,6302015,8.0,NaN,7012013,2015-12-31,4.6,NaN,2014-01-01,360082,EUCLID HOSPITAL,2017,NaN,9.0,No,NaN,3.0,OH,NaN,5.11,10,2017,5197.0,3038.0,2159.0,76300.0,76300.0,2.0,0.0,6.0,48.0,3.951,3.240,3.882,56.796,0.506201,0.000000,1.545595,0.845130,4.0,1.0,9.0,6.0,5.11,No,1,8.881784e-16,0.0,0.0,0.0,0.0
2204,NaN,9.0,NaN,2.0,NaN,3.0,NaN,9.0,6302015,9.0,NaN,7012013,2015-12-31,4.6,NaN,2014-01-01,360084,AULTMAN HOSPITAL,2017,NaN,5.0,No,NaN,4.0,OH,NaN,5.26,10,2017,26750.0,15786.0,10964.0,224014.0,199995.0,4.0,17.0,9.0,61.0,33.395,21.023,14.177,130.309,0.119778,0.808638,0.634831,0.468118,2.0,9.0,5.0,3.0,5.26,No,1,0.000000e+00,0.0,0.0,0.0,0.0
2205,NaN,8.0,NaN,8.0,NaN,4.0,NaN,6.0,6302015,8.0,NaN,7012013,2015-12-31,6.0,NaN,2014-01-01,360085,OHIO STATE UNIVERSITY STATE HEALTH SYSTEM,2017,NaN,6.0,No,NaN,6.0,OH,NaN,6.30,10,2017,87951.0,46810.0,41141.0,549053.0,508360.0,136.0,38.0,55.0,327.0,122.621,92.514,61.561,466.866,1.109109,0.410749,0.893423,0.700415,8.0,6.0,6.0,4.0,6.30,No,1,0.000000e+00,0.0,0.0,0.0,0.0
2206,NaN,3.0,NaN,2.0,NaN,8.0,NaN,2.0,6302015,3.0,NaN,7012013,2015-12-31,4.8,NaN,2014-01-01,360086,SPRINGFIELD REGIONAL MEDICAL CENTER,2017,NaN,10.0,No,NaN,2.0,OH,NaN,4.53,10,2017,22754.0,12443.0,10311.0,129444.0,123040.0,2.0,1.0,22.0,83.0,24.886,19.866,6.316,80.769,0.080366,0.050337,3.483217,1.027622,2.0,2.0,10.0,7.0,4.36,No,1,1.700000e-01,1.0,0.0,0.0,0.0


In [7]:
hac_scores = []
ct1 = 0
ct2 = 0

for hosp in df_2017['Facility ID'].tolist():
    tdf = df_2017[df_2017['Facility ID'] == hosp]

    d1 = 0
    d2 = 0
        
    w_ls = []
    sum_ls = []

    # Use original scores for all HAIs for a specific type of testing, i.e., can penalties be reproduced when using data directly from the HAC files
    m_ls = ['CAUTI derived score', 'CLABSI derived score', 'MRSA derived score', 'CDI derived score', 'SSI Score']
    
    # Use original SSI scores but derived scores for CDI, CAUTI, CLABSI, and MRSA for actual results
    #m_ls = ['CAUTI Score', 'CLABSI Score', 'MRSA Score', 'CDI Score', 'SSI Score']
                
    s = 0
    w = 0
    for m in m_ls:
        v = tdf[m].tolist()
        if len(list(set(v))) > 1:
            print('len(list(set(v))) > 1')
            sys.exit()

        v = tdf[m].iloc[0]

        if np.isnan(v) == False: 
            s += v
            w += 1

    if w == 0:
        d2 = np.nan
    else:
        d2 = s/w
        
    d1 = float(tdf['AHRQ PSI-90 Score'].iloc[0])
        
    d2_o = float(tdf['Domain 2 Score'].iloc[0])
    state = tdf['State'].iloc[0]
    
    if np.isnan(d1) == True: 
        # if no score for Domain 1, then total HAC score will be based entirely on Domain 2
        if np.isnan(d2) == True and np.isnan(d2_o) == True:
            hac_scores.append(d2)
        
        elif np.isnan(d2) == True and np.isnan(d2_o) == False:
            hac_scores.append(d2_o)
        
        elif np.isnan(d2) == False and np.isnan(d2_o) == False:
            hac_scores.append(d2)
        
        elif np.isnan(d2) == False and np.isnan(d2_o) == True:
            hac_scores.append(d2_o)
        
    elif np.isnan(d1) == False:
        # if there is a score for Domain 1 ...
        
        if state == 'MD':
            # states in MD should not have scores for domain 1
            print('Error:')
            print('Domain 1:', d1)
            print('State:', state)
            print('Hospitals in this state should not have domain 1 scores.\n')
            
        # Domain 2:
        
        # If the derived score is NaN and the original score is NaN ...
        if np.isnan(d2) == True and np.isnan(d2_o) == True:
            hac_scores.append(d1)
        
        # If the derived score is a float but the original score is NaN ...
        elif np.isnan(d2) == False and np.isnan(d2_o) == True:
            hac_scores.append(d1)

        # If the derived score is NaN but the original score is a float ...
        elif np.isnan(d2) == True and np.isnan(d2_o) == False:
            ct1 += 1
            hac_scores.append(0.15*d1 + 0.85*d2_o)
        
        # If the derived score is a float and the original score is a float ...
        elif np.isnan(d2) == False and np.isnan(d2_o) == False:
            if d2 != d2_o:
                ct2 += 1
                #print(d2, d2_o)
            hac_scores.append(0.15*d1 + 0.85*d2)
        
    else:
        print(d1, ',', d2, ',', d2_o)
        break

print('ct1:', ct1)
print('ct2:', ct2)


ct1: 1
ct2: 718


In [8]:
df_2017['Total HAC Score (derived)'] = hac_scores
print(df_2017.shape[0], 'hospitals in hac_df')

tdf = df_2017[~df_2017['Total HAC Score (derived)'].isin([np.nan, float('NaN')])]
p75 = np.percentile(tdf['Total HAC Score (derived)'], 75, method='linear')
p75 = 6.5700 # HACRP had modified its 75th percentile to this

pr = []
for hosp in df_2017['Facility ID'].tolist():
    tdf = df_2017[df_2017['Facility ID'] == hosp]

    p = tdf['Payment Reduction'].iloc[0]
    if p != 'Yes' and p != 'No' and np.isnan(p) == True:
        pr.append(np.nan)

    else:
        score = tdf['Total HAC Score (derived)'].iloc[0]

        if np.isnan(score) == True:
            pr.append('No')
        elif score <= p75:
            pr.append('No')
        elif score > p75:
            pr.append('Yes')
        else:
            print('This score is an error:', score)
            sys.exit()
            

df_2017['Payment Reduction (derived)'] = pr
            
o_list = df_2017['Payment Reduction'].tolist()
d_list = df_2017['Payment Reduction (derived)'].tolist()

same = 0
diff = 0
res_ls = []
for i, o in enumerate(o_list):
    if o == d_list[i]:
        same += 1
        res_ls.append(1)
    else:
        diff += 1
        res_ls.append(0)
            
df_2017['Payment Reduction Reproduced?'] = res_ls
    
print(same, "Penalty assignments were reproduced")
print(diff, "Penalty assignments were not reproduced")
print(str(np.round(100 * same/(same+diff),2)) + '% penalty assignments were reproduced\n')
    
df_2017['HAC delta'] = df_2017['Total HAC Score'] - df_2017['Total HAC Score (derived)']
df_2017['CDI delta'] = df_2017['CDI Score'] - df_2017['CDI derived score']
df_2017['MRSA delta'] = df_2017['MRSA Score'] - df_2017['MRSA derived score']
df_2017['CAUTI delta'] = df_2017['CAUTI Score'] - df_2017['CAUTI derived score']
df_2017['CLABSI delta'] = df_2017['CLABSI Score'] - df_2017['CLABSI derived score']


3202 hospitals in hac_df
3201 Penalty assignments were reproduced
1 Penalty assignments were not reproduced
99.97% penalty assignments were reproduced



In [9]:
display_df = df_2017[df_2017['Payment Reduction Reproduced?'] == 0]
items = ['Facility ID', 'file_year', 'HAI Measures End Date', 'HAI Measures Start Date',
             'CAUTI Footnote', 'CAUTI Score', 'CAUTI derived score',
             'CLABSI Footnote', 'CLABSI Score', 'CLABSI derived score',
             'CDI Footnote', 'CDI Score', 'CDI derived score',
             'MRSA Footnote', 'MRSA Score', 'MRSA derived score',
             'SSI Footnote', 'SSI Score',
             'AHRQ PSI-90 Footnote', 'AHRQ PSI-90 Score',
             'Total HAC Footnote', 'Total HAC Score', 'Total HAC Score (derived)',
             'Payment Reduction Footnote', 'Payment Reduction', 
             'Payment Reduction (derived)', 'Payment Reduction Reproduced?',
             ]

display_df = display_df.filter(items=items)
display_df = display_df[~display_df['Total HAC Score'].isin([float("NaN"), np.nan])]
display_df = display_df[~display_df['Total HAC Score (derived)'].isin([float("NaN"), np.nan])]
display_df = display_df.round(4)

display_df.sort_values(by=['CAUTI derived score',
                           'CLABSI derived score',
                           'CDI derived score',
                           'MRSA derived score',
                          ], ascending=False, inplace=True)
display_df.head(43)

,Facility ID,file_year,CAUTI Footnote,CAUTI Score,CAUTI derived score,CLABSI Footnote,CLABSI Score,CLABSI derived score,CDI Footnote,CDI Score,CDI derived score,MRSA Footnote,MRSA Score,MRSA derived score,SSI Footnote,SSI Score,AHRQ PSI-90 Footnote,AHRQ PSI-90 Score,Total HAC Footnote,Total HAC Score,Total HAC Score (derived),Payment Reduction,Payment Reduction (derived),Payment Reduction Reproduced?
1862,310086,2017,NaN,6.0,6.0,NaN,6.0,7.0,NaN,8.0,8.0,NaN,4.0,4.0,NaN,8.0,NaN,7.0,NaN,6.49,6.66,No,Yes,0


In [10]:
holdout_df = pd.read_pickle('~/GitHub/HACRP-HAIs/data/merged_HAC_HAI/P1_HAI_HAC_2017_holdout.pkl')

ls1 = list(df_2017)
ls2 = list(holdout_df)
ls = list(filter(lambda x:x in ls1, ls2))
print(df_2017.shape)
print(holdout_df.shape)
df_2017 = df_2017.merge(holdout_df, how='outer', on=ls)
print(df_2017.shape)


(3202, 58)
(112, 51)
(3314, 59)


In [11]:
print(sorted(holdout_df['State'].unique()))
print(sorted(df_2017['State'].unique()))

['AL', 'AR', 'AZ', 'CA', 'CO', 'FL', 'ID', 'IL', 'IN', 'KY', 'LA', 'MD', 'MN', 'MS', 'NC', 'ND', 'NM', 'NV', 'NY', 'OH', 'OK', 'SC', 'SD', 'TX', 'VA', 'WY']
['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']


In [12]:
tdf1 = df_2017[df_2017['Payment Reduction'] == 'Yes']
tdf2 = df_2017[df_2017['Payment Reduction'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_2017[df_2017['Payment Reduction'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])

print(df_2017.shape[0])
print(len(df_2017['Facility ID'].unique()))

df_2017.to_pickle('~/GitHub/HACRP-HAIs/data/merged_HAC_HAI/HAI_HAC_2017.pkl', protocol=5)



0.23507805325987144
0.7649219467401286
3314
3314
